In [29]:
import pandas as pd

data = pd.read_csv('../preprocess/202111171547_all.csv')
df = pd.DataFrame(data)
df = df.dropna()
data = df.iloc[:, :-1]
target = df['target']

In [30]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, stratify=target, random_state=0)
train_input, val_input, train_target, val_target = train_test_split(train_input, train_target, test_size=0.2, stratify=train_target, random_state=0)

In [31]:
from tensorflow import keras
import tensorflow.keras.backend as K

K.clear_session()
model = keras.Sequential()
model.add(keras.layers.SimpleRNN(15, activation="tanh", input_shape=(train_input.shape[1],1), return_sequences=True))
model.add(keras.layers.SimpleRNN(7, activation="tanh", dropout=0.3))
model.add(keras.layers.Dense(1, activation="relu"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 160, 15)           255       
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 7)                 161       
                                                                 
 dense (Dense)               (None, 1)                 8         
                                                                 
Total params: 424
Trainable params: 424
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

checkpoint_cb = keras.callbacks.ModelCheckpoint('11241505-best-lstm-model.h5')
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 15, restore_best_weights=True)

history = model.fit(train_input, train_target, 
                    epochs=100, batch_size = 50,
                    validation_data = (val_input, val_target),
                    callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/100
64/64 [==============================] - 4s 41ms/step - loss: 4.5625 - accuracy: 0.5535 - val_loss: 1.4981 - val_accuracy: 0.6440
Epoch 2/100
64/64 [==============================] - 2s 36ms/step - loss: 4.4390 - accuracy: 0.5472 - val_loss: 5.4909 - val_accuracy: 0.6440
Epoch 3/100
64/64 [==============================] - 2s 37ms/step - loss: 3.6820 - accuracy: 0.5651 - val_loss: 5.4909 - val_accuracy: 0.6440
Epoch 4/100
64/64 [==============================] - 2s 35ms/step - loss: 3.4369 - accuracy: 0.5809 - val_loss: 1.5416 - val_accuracy: 0.6440
Epoch 5/100
64/64 [==============================] - 2s 34ms/step - loss: 2.9024 - accuracy: 0.5636 - val_loss: 0.6520 - val_accuracy: 0.6440
Epoch 6/100
64/64 [==============================] - 2s 35ms/step - loss: 1.8854 - accuracy: 0.5532 - val_loss: 0.6573 - val_accuracy: 0.6440
Epoch 7/100
64/64 [==============================] - 2s 34ms/step - loss: 1.5055 - accuracy: 0.5135 - val_loss: 0.6725 - val_accuracy: 0.6491
Epoch 

In [33]:
model.evaluate(val_input, val_target)

25/25 [==============================] - 0s 9ms/step - loss: 0.5466 - accuracy: 0.7912


[0.5466133952140808, 0.7911949753761292]

In [34]:
model.evaluate(test_input, test_target)

32/32 [==============================] - 0s 8ms/step - loss: 0.5038 - accuracy: 0.8038


[0.5038419365882874, 0.803822934627533]